In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Face Detection Evaluation\n",
    "\n",
    "This notebook evaluates different face detection models and their performance on classroom video data.\n",
    "\n",
    "## Models to Test:\n",
    "- OpenCV Haar Cascade\n",
    "- MTCNN\n",
    "- RetinaFace\n",
    "- YOLO-based face detection\n",
    "\n",
    "## Metrics:\n",
    "- Detection accuracy\n",
    "- Processing speed (FPS)\n",
    "- False positive/negative rates"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('../src')\n",
    "\n",
    "import cv2\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import time\n",
    "from pathlib import Path\n",
    "import json\n",
    "\n",
    "from config import Config\n",
    "from detection.face_tracker import FaceTracker\n",
    "from utils.video_utils import VideoProcessor"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load configuration\n",
    "config = Config()\n",
    "video_processor = VideoProcessor(config)\n",
    "\n",
    "# Test video path\n",
    "video_path = \"../data/raw_videos/sample_classroom.mp4\"\n",
    "\n",
    "print(f\"Configuration loaded\")\n",
    "print(f\"Video path: {video_path}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Model Comparison"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def evaluate_detection_model(model_name, video_path, max_frames=100):\n",
    "    \"\"\"Evaluate a face detection model on video data.\"\"\"\n",
    "    \n",
    "    # Set model in config\n",
    "    config.set('face_detection.model', model_name)\n",
    "    \n",
    "    # Initialize tracker\n",
    "    tracker = FaceTracker(config)\n",
    "    \n",
    "    # Process video\n",
    "    cap = cv2.VideoCapture(video_path)\n",
    "    if not cap.isOpened():\n",
    "        print(f\"Failed to open video: {video_path}\")\n",
    "        return None\n",
    "    \n",
    "    detections = []\n",
    "    frame_count = 0\n",
    "    start_time = time.time()\n",
    "    \n",
    "    while frame_count < max_frames:\n",
    "        ret, frame = cap.read()\n",
    "        if not ret:\n",
    "            break\n",
    "        \n",
    "        # Detect faces\n",
    "        frame_detections = tracker.detect_faces(frame)\n",
    "        \n",
    "        # Add frame info\n",
    "        for detection in frame_detections:\n",
    "            detection['frame_idx'] = frame_count\n",
    "            detection['model'] = model_name\n",
    "        \n",
    "        detections.extend(frame_detections)\n",
    "        frame_count += 1\n",
    "    \n",
    "    cap.release()\n",
    "    end_time = time.time()\n",
    "    \n",
    "    # Calculate metrics\n",
    "    processing_time = end_time - start_time\n",
    "    fps = frame_count / processing_time\n",
    "    total_detections = len(detections)\n",
    "    \n",
    "    return {\n",
    "        'model': model_name,\n",
    "        'total_detections': total_detections,\n",
    "        'frames_processed': frame_count,\n",
    "        'processing_time': processing_time,\n",
    "        'fps': fps,\n",
    "        'detections_per_frame': total_detections / frame_count if frame_count > 0 else 0,\n",
    "        'detections': detections\n",
    "    }"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test different models\n",
    "models_to_test = ['opencv', 'mtcnn', 'retinaface', 'yolo']\n",
    "results = []\n",
    "\n",
    "for model in models_to_test:\n",
    "    print(f\"Testing {model}...\")\n",
    "    try:\n",
    "        result = evaluate_detection_model(model, video_path)\n",
    "        if result:\n",
    "            results.append(result)\n",
    "            print(f\"  FPS: {result['fps']:.2f}\")\n",
    "            print(f\"  Total detections: {result['total_detections']}\")\n",
    "    except Exception as e:\n",
    "        print(f\"  Error testing {model}: {e}\")\n",
    "\n",
    "print(f\"\\nCompleted testing {len(results)} models\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Results Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create comparison plots\n",
    "if results:\n",
    "    models = [r['model'] for r in results]\n",
    "    fps_values = [r['fps'] for r in results]\n",
    "    detection_counts = [r['total_detections'] for r in results]\n",
    "    \n",
    "    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))\n",
    "    \n",
    "    # FPS comparison\n",
    "    ax1.bar(models, fps_values, color='skyblue')\n",
    "    ax1.set_title('Processing Speed (FPS)')\n",
    "    ax1.set_ylabel('Frames per Second')\n",
    "    ax1.tick_params(axis='x', rotation=45)\n",
    "    \n",
    "    # Detection count comparison\n",
    "    ax2.bar(models, detection_counts, color='lightcoral')\n",
    "    ax2.set_title('Total Detections')\n",
    "    ax2.set_ylabel('Number of Detections')\n",
    "    ax2.tick_params(axis='x', rotation=45)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Print detailed results\n",
    "    print(\"\\nDetailed Results:\")\n",
    "    print(\"-\" * 60)\n",
    "    for result in results:\n",
    "        print(f\"Model: {result['model']}\")\n",
    "        print(f\"  FPS: {result['fps']:.2f}\")\n",
    "        print(f\"  Total Detections: {result['total_detections']}\")\n",
    "        print(f\"  Detections/Frame: {result['detections_per_frame']:.2f}\")\n",
    "        print(f\"  Processing Time: {result['processing_time']:.2f}s\")\n",
    "        print()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Frame-by-Frame Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze detection patterns over time\n",
    "if results:\n",
    "    best_model = max(results, key=lambda x: x['fps'])\n",
    "    \n",
    "    # Group detections by frame\n",
    "    frame_detections = {}\n",
    "    for detection in best_model['detections']:\n",
    "        frame_idx = detection['frame_idx']\n",
    "        if frame_idx not in frame_detections:\n",
    "            frame_detections[frame_idx] = []\n",
    "        frame_detections[frame_idx].append(detection)\n",
    "    \n",
    "    # Plot detections over time\n",
    "    frames = sorted(frame_detections.keys())\n",
    "    detection_counts = [len(frame_detections[frame]) for frame in frames]\n",
    "    \n",
    "    plt.figure(figsize=(12, 6))\n",
    "    plt.plot(frames, detection_counts, 'b-', linewidth=2)\n",
    "    plt.title(f'Face Detections Over Time - {best_model[\"model\"]}')\n",
    "    plt.xlabel('Frame Number')\n",
    "    plt.ylabel('Number of Faces Detected')\n",
    "    plt.grid(True, alpha=0.3)\n",
    "    plt.show()\n",
    "    \n",
    "    print(f\"Best performing model: {best_model['model']}\")\n",
    "    print(f\"Average faces per frame: {np.mean(detection_counts):.2f}\")\n",
    "    print(f\"Max faces in a frame: {max(detection_counts)}\")\n",
    "    print(f\"Min faces in a frame: {min(detection_counts)}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Confidence Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze confidence distributions\n",
    "if results:\n",
    "    fig, axes = plt.subplots(2, 2, figsize=(12, 10))\n",
    "    axes = axes.flatten()\n",
    "    \n",
    "    for i, result in enumerate(results[:4]):  # Show up to 4 models\n",
    "        confidences = [d['confidence'] for d in result['detections']]\n",
    "        \n",
    "        axes[i].hist(confidences, bins=20, alpha=0.7, color='steelblue')\n",
    "        axes[i].set_title(f'{result[\"model\"]} - Confidence Distribution')\n",
    "        axes[i].set_xlabel('Confidence Score')\n",
    "        axes[i].set_ylabel('Frequency')\n",
    "        axes[i].grid(True, alpha=0.3)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Print confidence statistics\n",
    "    print(\"\\nConfidence Statistics:\")\n",
    "    print(\"-\" * 40)\n",
    "    for result in results:\n",
    "        confidences = [d['confidence'] for d in result['detections']]\n",
    "        if confidences:\n",
    "            print(f\"{result['model']}:\")\n",
    "            print(f\"  Mean: {np.mean(confidences):.3f}\")\n",
    "            print(f\"  Std: {np.std(confidences):.3f}\")\n",
    "            print(f\"  Min: {min(confidences):.3f}\")\n",
    "            print(f\"  Max: {max(confidences):.3f}\")\n",
    "            print()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Save Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save evaluation results\n",
    "if results:\n",
    "    output_path = Path('../data/outputs')\n",
    "    output_path.mkdir(parents=True, exist_ok=True)\n",
    "    \n",
    "    # Save summary results\n",
    "    summary_results = []\n",
    "    for result in results:\n",
    "        summary_results.append({\n",
    "            'model': result['model'],\n",
    "            'fps': result['fps'],\n",
    "            'total_detections': result['total_detections'],\n",
    "            'detections_per_frame': result['detections_per_frame'],\n",
    "            'processing_time': result['processing_time']\n",
    "        })\n",
    "    \n",
    "    with open(output_path / 'detection_evaluation_results.json', 'w') as f:\n",
    "        json.dump(summary_results, f, indent=2)\n",
    "    \n",
    "    print(f\"Results saved to {output_path / 'detection_evaluation_results.json'}\")\n",
    "    \n",
    "    # Save detailed detections for best model\n",
    "    best_model = max(results, key=lambda x: x['fps'])\n",
    "    with open(output_path / f'detections_{best_model[\"model\"]}.json', 'w') as f:\n",
    "        json.dump(best_model['detections'], f, indent=2)\n",
    "    \n",
    "    print(f\"Detailed detections saved to {output_path / f'detections_{best_model['model']}.json'}\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}